In [5]:
import requests
import os


base_url_of_cluster_scaler = "http://ddl-cluster-scaler-svc.domino-field.svc.cluster.local"
service_name="ddl_cluster_scaler"

def is_cluster_auto_scaler_healthy():
    # Is service running
    healthz_url = f"{base_url_of_cluster_scaler}/healthz"
    result = requests.get(healthz_url)
    return result.status_code, result.text

In [6]:
is_cluster_auto_scaler_healthy()

(200, '{"status":"healthy"}\n')

In [7]:
def get_auth_headers():
    domino_api_proxy= os.environ['DOMINO_API_PROXY']
    #This is a proxy for the service account tokem
    token = requests.get(f"{domino_api_proxy}/access-token").text
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"   # add this if you're posting JSON
    }
    return headers 

In [ ]:
def get_cluster_status(cluster_kind:str="rayclusters"):  
    cluster_id_prefix = cluster_kind.replace("clusters","")
    cluster_id = f"{cluster_id_prefix}-{os.environ['DOMINO_RUN_ID']}"
    print(cluster_id)
    get_cluster_url = f"{base_url_of_cluster_scaler}/{service_name}/get/{cluster_kind}/{cluster_id}"
    print(get_cluster_url)
    resp = requests.get(get_cluster_url,headers=get_auth_headers())
    print(f"Status code {resp.status_code}")
    cluster_details = resp.json()
    return cluster_details



In [ ]:
def scale_cluster(cluster_kind:str="rayclusters",replicas:int=1):
    cluster_id_prefix = cluster_kind.replace("clusters","")
    cluster_id = f"{cluster_id_prefix}-{os.environ['DOMINO_RUN_ID']}"
    scale_cluster_url = f"{base_url_of_cluster_scaler}/{service_name}/scale/{cluster_kind}/{cluster_id}"
    print(scale_cluster_url)
    
    resp = requests.post(
        scale_cluster_url,
        headers=get_auth_headers(),
        json={"replicas": replicas},
        timeout=(3.05, 10),
    )
    print(f"Status code {resp.status_code}")
    cluster_details = resp.json()
    return cluster_details